# Paso 1: Implementación de la anotación

Federico Ortega Riba

## 1. Fase de preparación y preprocesamiento

In [ ]:
import os
import xml.etree.ElementTree as ET
import re

In [ ]:
from google.colab import drive

# Acceso a través de Google Drive
drive.mount('/content/drive')

# Indicamos la ruta de la carpeta en Google Drive con nuestros textos del BOE y de EurLex
path = "/content/drive/MyDrive/Aplicaciones/Doaa/Textos/"
carpeta_final = "/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Extraidos/"

Mounted at /content/drive


### 1.1 extracción datos .XML

In [ ]:
def extraccion_texto_xml(ruta, carpeta_final):
    texto = []

    for archivo in os.listdir(ruta):
        if archivo.endswith(".xml"):
            ruta_completa = os.path.join(ruta, archivo)
            tree = ET.parse(ruta_completa)

            texto_completo = ""
            for elemento in tree.getroot().iter():
                if elemento.tag in ["p", "em", "head", "title"]:
                    texto_completo += ET.tostring(elemento, encoding="unicode", method="xml")
            texto_completo = re.sub(r"<[^>]*>", "", texto_completo)
            texto.append(texto_completo.strip())

            # Escribir el texto extraído en un archivo TXT con el mismo nombre que el archivo de entrada
            nombre_txt = os.path.splitext(archivo)[0] + ".txt"
            with open(os.path.join(carpeta_final, nombre_txt), "w") as f:
                f.write(texto_completo.strip())

    return texto

# Llamamos a la función y le pasamos la ruta de la carpeta con los archivos XML
textos_extraidos_xml = extraccion_texto_xml(path, carpeta_final)

In [ ]:
# Imprimimos el primer texto extraído de los documentos XML
if textos_extraidos_xml:
    texto_1_xml = textos_extraidos_xml[0]
    print("Primer texto del BOE/JRC extraído:")
    print(texto_1_xml )
else:
    print("No se ha extraído ningún texto del XML proporcionado.")

Primer texto del BOE/JRC extraído:
JRC-ACQUIS 22005A0607(01) Spanish
        Acuerdo de cooperaci%oacute%n entre la Comunidad Europea de la Energ%iacute%a At%oacute%mica y el Gobierno de la Rep%uacute%blica de Kazajst%aacute%n en el %aacute%mbito de la fusi%oacute%n termonuclear controlada
      Acuerdo de cooperaci%oacute%n entre la Comunidad Europea de la Energ%iacute%a At%oacute%mica y el Gobierno de la Rep%uacute%blica de Kazajst%aacute%n en el %aacute%mbito de la fusi%oacute%n termonuclear controlada
      Acuerdo
        de cooperación entre la Comunidad Europea de la Energía Atómica y el Gobierno de la República de Kazajstán en el ámbito de la fusión termonuclear controlada
        El GOBIERNO DE LA REPÚBLICA DE KAZAJSTÁN, en lo sucesivo denominado "Kazajstán",
        por una parte, y
        LA COMUNIDAD EUROPEA DE LA ENERGÍA ATÓMICA, en lo sucesivo denominada "la Comunidad",
        por otra,
        ambos también denominados por lo general en lo sucesivo "la Parte" o "las Pa

### 1.2 extracción datos HTML

In [ ]:
from bs4 import BeautifulSoup

# Ahora vamos a extraer el texto de los documentos que tenemos en HTML

def extraccion_texto_html(ruta, carpeta_final):
    texto = []

    # Tenemos que crear una carpeta de destino si no existe ya
    if not os.path.exists(carpeta_final):
        os.makedirs(carpeta_final)

    for archivo in os.listdir(ruta):
        if archivo.endswith(".html"):
            ruta_completa = os.path.join(ruta, archivo)
            with open(ruta_completa, "r", encoding="utf-8") as file:
                texto_html = file.read()

            # Pasamos nuestro texto por BeautifulSoup
            soup = BeautifulSoup(texto_html, "html.parser")

            # Extraemos el texto de las etiquetas <p>
            texto_completo = ""
            for elemento in soup.find_all("p"):
                if elemento.text:
                    texto_completo += elemento.text.strip() + "\n"

            # Eliminamos las etiquetas <p> del HTML
            texto_completo = re.sub(r"<[^>]*>", "", texto_completo)

            # El texto que hayamos extraído lo pasamos en txt a nuestra carpeta_final
            nombre_archivo_txt = os.path.splitext(archivo)[0] + ".txt"
            ruta_destino = os.path.join(carpeta_final, nombre_archivo_txt)
            with open(ruta_destino, "w", encoding="utf-8") as f:
                f.write(texto_completo.strip())

            texto.append(texto_completo.strip())

    return texto

# Llamar a la función y pasarle la ruta de la carpeta que contiene los archivos XML
textos_extraidos_html = extraccion_texto_html(path, carpeta_final)

In [ ]:
# Imprimimos el primer texto de los HTML que hemos extraido
if textos_extraidos_html:
    texto_1_html = textos_extraidos_html[0]
    print("Primer texto del EurLex extraído:")
    print(texto_1_html)
else:
    print("No se ha extraído ningún texto del HTML proporcionado.")

Primer texto del EurLex extraído:
Avis juridique important
Protocolo de Ginebra (1979), anexo al Acuerdo General sobre Aranceles Aduaneros y Comercio  


Diario Oficial n° L 071 de 17/03/1980 p. 0003 - 0004 Edición especial en finés : Capítulo 11 Tomo 9 p. 0005  Edición especial sueca: Capítulo 11 Tomo 9 p. 0005  Edición especial griega: Capítulo 11 Tomo 19 p. 0005  Edición especial en español: Capítulo 11 Tomo 12 p. 0040  Edición especial en portugués: Capítulo 11 Tomo 12 p. 0040
PROTOCOLO DE GINEBRA ( 1979 ) , ANEXO AL ACUERDO   GENERAL SOBRE ARANCELES ADUANEROS Y COMERCIO    LAS PARTES CONTRATANTES DEL ACUERDO GENERAL SOBRE   ARANCELES ADUANEROS Y COMERCIO Y LA COMUNIDAD   ECONÓMICA EUROPEA que han participado en las   Negociaciones Comerciales Multilaterales de 1973-1979   ( denominadas en adelante « los participantes » ) ,    HABIENDO llevado a cabo negociaciones de conformidad   con el artículo XXVIII bis , el artículo XXXIII   y demás disposiciones pertinentes del Acuerdo Genera

### 1.3. Normalización de los textos

Podemos normalizar el texto quitando los diacríticos o con ellos. En este caso, lo vamos a hacer con diacríticos.

In [ ]:
def limpiar_texto(texto):
    # Eliminamos espacios extras y espacios al inicio y final
    texto_limpio = ' '.join(texto.split())
    # Reemplazamos caracteres mal codificados, menos las letras acentuadas
    texto_limpio = re.sub(r'[^\x00-\x7FáéíóúÁÉÍÓÚüÜñÑ]+', ' ', texto_limpio)
    # Los espacios múltiples pasan a ser solo uno
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio)
    # Eliminamos espacios al inicio y final
    texto_limpio = texto_limpio.strip()
    return texto_limpio

def reescribir_textos_normalizados(carpeta_final):
    texto_normalizado = []

    for archivo in os.listdir(carpeta_final):
        if archivo.endswith(".txt"):
            ruta_archivo = os.path.join(carpeta_final, archivo)

            # Leemos el contenido de nuestro archivo original
            with open(ruta_archivo, "r", encoding="utf-8") as file:
                contenido = file.read()

            # Normalizamos el contenido del archivo utilizando la función que ya hemos creado (limpiar_texto)
            contenido_normalizado = limpiar_texto(contenido)

            # Escribimos el contenido normalizado en el mismo archivo original
            with open(ruta_archivo, "w", encoding="utf-8") as file:
                file.write(contenido_normalizado)

            # Agregamos el texto normalizado a la lista
            texto_normalizado.append(contenido_normalizado)

    return texto_normalizado

In [ ]:
textos_normalizados = reescribir_textos_normalizados(carpeta_final)

In [ ]:
# Imprimimos el primer texto normalizado
if textos_normalizados:
    texto_norm_1 = textos_normalizados[0]
    print("Primer texto normalizado:")
    print(texto_norm_1)
else:
    print("No se ha extraído ningún texto normalizado.")

Primer texto normalizado:
JRC-ACQUIS 22005A0617(01) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la

### 1.4. Segmentar y tokenizar usando SpaCy

In [ ]:
!pip install -U spacy

In [ ]:
# Descargamos el modelo large en español
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Comprobamos la compatibilidad entre la versión de SpaCy de de los modelos
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.4) =================
ℹ spaCy installation: /usr/local/lib/python3.10/dist-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_lg   >=3.7.0,<3.8.0   3.7.0   ✔
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔



In [ ]:
import spacy

# Almacenamos en una variable el modelo que hemos cargado
nlp = spacy.load("es_core_news_lg")

In [ ]:
textos_tokenizados = []

# Procesamos cada texto normalizado pasándole el modelo de spaCy y lo añadimos a la lista de textos tokenizados
for texto in textos_normalizados:
    # Le pasamos spaCy
    doc = nlp(texto)
    # Añadimos los documentos que hemos procesado en doc a la lista de textos tokenizados
    textos_tokenizados.append(doc)

In [ ]:
# Comprobamos lo que nos ha devuelto haciendo un print del primer documento

texto_1_token = textos_tokenizados[0]

# Recorremos cada oración en el primer texto
for i, oracion in enumerate(texto_1_token.sents, start=1):
    # Imprimimos el número de la línea y la oración
    print(f"Línea {i}: {oracion.text}")

Línea 1: JRC-ACQUIS 22005A0617(01) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica

In [ ]:
# Definimos una función para extraer las estadísticas de cada texto

def estadistica(doc):
    # Número de tokens
    num_tokens = len(doc)

    # Numero de types (tokens únicos)
    types = len(set(token.text for token in doc))

    # Media de longitud de las oraciones
    longitud_oraciones = [len(sent) for sent in doc.sents]
    media_longitud_oraciones = sum(longitud_oraciones) / len(longitud_oraciones)

    return num_tokens, types, media_longitud_oraciones

# Recorremos cada texto tokenizado
for i, doc in enumerate(textos_tokenizados, start=1):
    print("Texto", i)
    num_tokens, types, media_longitud_oraciones = estadistica(doc)
    print("Número de tokens:", num_tokens)
    print("Número de types:", types)
    print("Media de longitud de oraciones: {:.2f}".format(media_longitud_oraciones))
    print("-" * 50)  # Separador entre textos


Texto 1
Número de tokens: 4066
Número de types: 1284
Media de longitud de oraciones: 40.26
--------------------------------------------------
Texto 2
Número de tokens: 1712
Número de types: 415
Media de longitud de oraciones: 27.61
--------------------------------------------------
Texto 3
Número de tokens: 3616
Número de types: 381
Media de longitud de oraciones: 44.10
--------------------------------------------------
Texto 4
Número de tokens: 3320
Número de types: 915
Media de longitud de oraciones: 46.11
--------------------------------------------------
Texto 5
Número de tokens: 3820
Número de types: 995
Media de longitud de oraciones: 38.59
--------------------------------------------------
Texto 6
Número de tokens: 401
Número de types: 157
Media de longitud de oraciones: 57.29
--------------------------------------------------
Texto 7
Número de tokens: 17560
Número de types: 2832
Media de longitud de oraciones: 20.90
--------------------------------------------------
Texto 8
Núm

## 2. Anotación

### 2.1 anotar las organizaciones mediante la anotación de NERC de spaCy

In [ ]:
#  Definimos una función para extraer las organizaciones usando los modelos de spaCy

def anotar_spaCy_ORG(textos_tokenizados):
    lista_tuplas_entidades = []

    for doc in textos_tokenizados:
        for sent in doc.sents:
            texto_sent = sent.text
            dict_entities = {}

            if sent.ents:
                lista_tupla_ne_spacy = []

                # Me he dado cuenta de que me reconoce los Reales Decretos como ORG,
                # así que le digo que no lo tenga en cuenta
                for ent in sent.ents:
                    if ent.label_ == "ORG" and "Real Decreto" not in ent.text:
                        tupla_ne_spacy = (ent.text, ent.start_char, ent.end_char, ent.label_)
                        lista_tupla_ne_spacy.append(tupla_ne_spacy)

                dict_entities["entities"] = lista_tupla_ne_spacy
                tupla_oración_entidades = (sent.text, dict_entities)
                lista_tuplas_entidades.append(tupla_oración_entidades)

    return lista_tuplas_entidades

In [ ]:
# Vamos a sacar las entidades de todos los textos

resultados_org = anotar_spaCy_ORG(textos_tokenizados)

In [ ]:
# Creamos una lista con las entidades del primer texto para poder visualizarlas mejor

entidades_texto_1 = []

if resultados_org:
    entidades_primer_texto = resultados_org[0][1]["entities"]
    for entidad in entidades_primer_texto:
        entidades_texto_1.append((" ".join(resultados_org[0][0].split()), {"entities": [entidad]}))
else:
    entidades_texto_1.append(("No se encontraron entidades en el primer texto.", {}))

# Imprimimos la lista de entidades
print(entidades_texto_1)

[('JRC-ACQUIS 22005A0617(01) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hu

In [ ]:
# Vamos a visualizar los datos de una manera más estética usando pandas

import pandas as pd

columnas = ["Oración", "Entidad", "Comienzo car.", "Fin car.", "Tipo"]
lista_resultados_procesados = []

for idx, (oracion, entidades) in enumerate(resultados_org):
    for entidad in entidades["entities"]:
        lista_resultados_procesados.append((oracion, entidad[0], entidad[1], entidad[2], entidad[3]))

# Creamos DataFrame en pandas
df_resultados_org = pd.DataFrame(lista_resultados_procesados, columns=columnas)

# Imprimimos DataFrame
df_resultados_org

,Oración,Entidad,Comienzo car.,Fin car.,Tipo
0,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Acuerdo Europeo,57,72,ORG
1,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Acuerdo Europeo,641,656,ORG
2,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Acuerdo Europeo,1225,1240,ORG
3,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Unión Europea de la República Checa,1421,1456,ORG
4,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,ALEMANIA,1769,1777,ORG
...,...,...,...,...,...
3121,El presente Acuerdo será depositado en poder d...,Partes Contratantes del Acuerdo General,91411,91450,ORG
3122,El presente Acuerdo será registrado de conform...,Carta de las Naciones Unidas,91853,91881,ORG
3123,(16) El término diferencias se usa en el GATT ...,GATT,95202,95206,ORG
3124,"(17) A este respecto , el Comité podrá señalar...",Comité,95367,95373,ORG


In [ ]:
# Vemos el número de entidades totales

print(f"Número Total de Entidades ORG: {len(df_resultados_org)}")

Número Total de Entidades ORG: 3126


## 2.2. Extraer las entidades de leyes mediante patrones de expresiones regulares

In [ ]:
import re

def extraer_leyes(textos_tokenizados):
    # Patrones para reconocer leyes, decisiones, reglamentos, tratados y órdenes
    patron_ley = re.compile(r'\b(?:Ley|Real\sDecreto)\b[^,]*(?:,\s*de[^,]*)?(?:,\s*(?:(?!se\s(publican|publica|anuncian|anuncia|notifican|notifica|comunican|comunica|informan|informa)\b)[^,])*)?')
    patron_decision = re.compile(r"Decisión.*?(?=\(DO|$)")
    patron_reglamento = re.compile(r"Reglamento.*?(?=\.|$)")
    patron_tratado = re.compile(r"Tratado constitutivo.*?(?=,|$)")
    patron_orden = re.compile(r"Orden\s(?:[A-Z\d\/]+(?:\/\d{4})?|(?:de\s)?\d{1,2}\sde\s[a-zA-Z]+\sde\s\d{4})")

    # Lista de tuplas para almacenar resultados
    lista_tuplas_leyes = []

    # Iteramos sobre textos tokenizados
    for doc in textos_tokenizados:
        for sent in doc.sents:
            oracion = sent.text
            leyes_encontradas = patron_ley.findall(oracion)
            decision_encontrada = patron_decision.findall(oracion)
            reglamento_encontrado = patron_reglamento.findall(oracion)
            tratado_encontrado = patron_tratado.findall(oracion)
            orden_encontrada = patron_orden.findall(oracion)

            if leyes_encontradas or decision_encontrada or reglamento_encontrado or tratado_encontrado or orden_encontrada:
                dict_entities = {"entities": [(ley, oracion.find(ley), oracion.find(ley) + len(ley), 'LEGAL') for ley in leyes_encontradas]}
                dict_entities["entities"].extend([(dec, oracion.find(dec), oracion.find(dec) + len(dec), 'LEGAL') for dec in decision_encontrada])
                dict_entities["entities"].extend([(reg, oracion.find(reg), oracion.find(reg) + len(reg), 'LEGAL') for reg in reglamento_encontrado])
                dict_entities["entities"].extend([(tra, oracion.find(tra), oracion.find(tra) + len(tra), 'LEGAL') for tra in tratado_encontrado])
                dict_entities["entities"].extend([(ord, oracion.find(ord), oracion.find(ord) + len(ord), 'LEGAL') for ord in orden_encontrada])

                tupla_resultado = (oracion, dict_entities)
                lista_tuplas_leyes.append(tupla_resultado)

    return lista_tuplas_leyes

# Llamamos a la función
leyes_en_textos = extraer_leyes(textos_tokenizados)

# Imprimimos resultados
print("Oraciones con entidades legales encontradas:")
for tupla in leyes_en_textos:
    print(tupla)


Oraciones con entidades legales encontradas:
('JRC-ACQUIS 22005A0617(01) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacut

In [ ]:
# Vemos el número de entidades totales

print(f"Número Total de Entidades LEGAL: {len(leyes_en_textos)}")

Número Total de Entidades LEGAL: 281


Ahora tenemos que combinar la función que hemos sacado para las entidades ORG y la que hemos sacado para las leyes y reales decretos con RegEx, así que definimos otra función que recoja las dos.

In [ ]:
def combinar_entidades(org_entidades, ley_entidades):
    # Creamos un diccionario para organizar las entidades por oración
    oraciones_entidades = {}

    # Agregamos entidades ORG al diccionario
    for oracion, entidades in org_entidades:
        oraciones_entidades.setdefault(oracion, {"entities": []})["entities"].extend(entidades.get("entities", []))

    # Agregamos entidades de ley al diccionario
    for oracion, entidades in ley_entidades:
        oraciones_entidades.setdefault(oracion, {"entities": []})["entities"].extend(entidades["entities"])

    # Convertimos el diccionario a una lista de tuplas
    lista_tuplas_resultado = [(oracion, {"entities": entidades["entities"]}) for oracion, entidades in oraciones_entidades.items()]

    return lista_tuplas_resultado

# Llamamos a la función para combinar entidades ORG y LEGAL
entidades_combinadas = combinar_entidades(resultados_org, leyes_en_textos)

# Imprimimos resultados
print("Entidades combinadas en cada oración:")
for tupla in entidades_combinadas:
    print(tupla)

Se han truncado las últimas 5000 líneas del flujo de salida.
('X=369088.8789 Y=3153479.9749 X=369014.6018 Y=3153434.8489 X=368988.7156 Y=3153417.3582 X=368956.9992 Y=3153396.9525 X=368928.8975 Y=3153369.6670 X=368917.2371 Y=3153355.4413 X=368901.6704 Y=3153341.7402 X=368877.8831 Y=3153309.6740 X=368849.8980 Y=3153284.9538 X=368834.9726 Y=3153273.6432 X=368815.9078 Y=3153263.2071 X=368799.9330 Y=3153256.3274 X=368779.5272 Y=3153248.9814 X=368732.0691 Y=3153233.7062 X=368706.8826 Y=3153220.2966 X=368606.2529 Y=3153161.9944 X=368541.0127 Y=3153119.9585 X=368511.5117 Y=3153116.3437 X=368477.5798 Y=3153113.4286 X=368441.2575 Y=3153101.9431 X=368422.6008 Y=3153095.7630 X=368391.2342 Y=3153083.4030 X=368370.1288 Y=3153057.1670 X=368352.9879 Y=3153031.2808 X=368329.0257 Y=3153013.1488 X=368312.5845 Y=3153009.5340 X=368297.3093 Y=3153004.8698 X=368283.0835 Y=3152999.3894 X=368271.7729 Y=3152990.2943 X=368254.6321 Y=3152972.3372 X=368243.3214 Y=3152960.6767 X=368225.4809 Y=3152948.6665 X=368190.

In [ ]:
# Ruta donde se guardarán los archivos anotados
path = "/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/"

# Compilamos los patrones regex
patron_ley = re.compile(r'\b(?:Ley|Real\sDecreto)\b[^,]*(?:,\s*de[^,]*)?(?:,\s*(?:(?!se\s(publican|publica|anuncian|anuncia|notifican|notifica|comunican|comunica|informan|informa)\b)[^,])*)?')
patron_decision = re.compile(r"Decisión.*?(?=\(DO|$)")
patron_reglamento = re.compile(r"Reglamento.*?(?=\.|$)")
patron_tratado = re.compile(r"Tratado constitutivo.*?(?=,|$)")
patron_orden = re.compile(r"Orden\s(?:[A-Z\d\/]+(?:\/\d{4})?|(?:de\s)?\d{1,2}\sde\s[a-zA-Z]+\sde\s\d{4})")

# Iteramos sobre cada documento tokenizado de la lista de textos tokenizados
for i, doc in enumerate(textos_tokenizados, start=1):
    # Creamos una lista para almacenar las tuplas de cada línea
    lineas_anotadas = []

    # Iteramos sobre cada oración del documento
    for sent in doc.sents:
        # Texto de la oración
        texto_oracion = sent.text

        # Buscamos todas las coincidencias de cada patrón en la oración actual
        matches_legales = re.finditer(patron_ley, texto_oracion)
        matches_decision = re.finditer(patron_decision, texto_oracion)
        matches_reglamento = re.finditer(patron_reglamento, texto_oracion)
        matches_tratado = re.finditer(patron_tratado, texto_oracion)
        matches_orden = re.finditer(patron_orden, texto_oracion)

        # Lista para almacenar todas las entidades encontradas en la oración actual
        entidades_oracion = []

        # Iteramos sobre las coincidencias encontradas para cada patrón
        for matches, etiqueta in [(matches_legales, "LEGAL"), (matches_decision, "LEGAL"), (matches_reglamento, "LEGAL"), (matches_tratado, "LEGAL"), (matches_orden, "LEGAL")]:
            for match in matches:
                entidad_texto = match.group()
                inicio, fin = match.span()
                entidades_oracion.append((entidad_texto, inicio, fin, etiqueta))

        # Obtenemos las entidades ORG de la oración actual usando la función proporcionada
        entidades_org = anotar_spaCy_ORG([sent])

        # Verificamos si se encontraron entidades ORG en la oración
        if entidades_org:
            # Agregamos las entidades ORG de la oración actual a la lista de datos de entrenamiento
            lineas_anotadas.append(entidades_org[0])

        # Verificamos si se encontraron entidades en la oración
        if entidades_oracion:
            # Ordenamos las entidades por su posición de inicio
            entidades_oracion.sort(key=lambda x: x[1])
            # Agregamos las entidades encontradas de la oración actual a la lista de datos de entrenamiento
            lineas_anotadas.append((texto_oracion, {"entities": [(entidad_texto, inicio, fin, etiqueta) for entidad_texto, inicio, fin, etiqueta in entidades_oracion]}))

    # Guardamos las líneas anotadas en un nuevo archivo .txt
    nombre_archivo = os.path.join(path, f"doc_{i}_anotado.txt")
    with open(nombre_archivo, 'w', encoding="utf-8") as archivo:
        # Escribimos las tuplas en el archivo
        for linea_anotada in lineas_anotadas:
            archivo.write(f"{linea_anotada}\n")

    print(f"Se ha creado el archivo '{nombre_archivo}'.")

Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_1_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_2_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_3_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_4_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_5_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_6_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_7_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_8_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_9_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicac

Pasamos ahora a crear automáticamente los documentos txt donde almacenaremos nuestra lista de tuplas con el formato deseado

In [ ]:
# Vamos a comprobar cómo queda el archivo final

# Definimos la ruta del archivo a imprimir
doc_anotado_1 = "/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_1_anotado.txt"

# Leemos e imprimimos el contenido del archivo
with open(doc_anotado_1, 'r') as file:
    contenido = file.read()
    print("Contenido del primer archivo anotado:")
    print(contenido)

Contenido del primer archivo anotado:
('Avis juridique important Acuerdos multilaterales resultantes de las negociaciones comerciales de 1973-1979 (GATT) - Protocolo del acuerdo relativo a la aplicación del artículo VII del Acuerdo General sobre Aranceles Aduaneros y Comercio Diario Oficial n L 071 de 17/03/1980 p. 0127 - 0128 Diario Oficial n L 071 de 17/03/1980 p. 0127 - 0128 Edición especial en finés : Capítulo 11 Tomo 9 p. 0129 Edición especial sueca: Capítulo 11 Tomo 9 p. 0129 Edición especial griega: Capítulo 11 Tomo 19 p. 0130 Edición especial en español: Capítulo 11 Tomo 12 p. 0163 Edición especial en portugués: Capítulo 11 Tomo 12 p. 0163 PROTOCOLO DEL ACUERDO RELATIVO A LA APLICACIÓN DEL ARTÍCULO VII DEL ACUERDO GENERAL SOBRE ARANCELES ADUANEROS Y COMERCIO Las Partes en el Acuerdo relativo a la aplicación del artículo VII del Acuerdo General sobre Aranceles Aduaneros y Comercio ( denominado en adelante Acuerdo ) , Tomando en consideración las negociaciones comerciales multila

# Paso 3: Validación del Silver Standard

Federico Ortega Riba

### Opción 1 (Volcar Silver Standard en un Excel)

In [ ]:
# Con la salida de la Tarea 5B, recurrir la lista para crear un DataFrame en pandas con estas columnas:
# Texto de la oración, Texto de la entidad detectada, Tipo de la entidad, Posición_inicio, Posición_fin

In [ ]:
import pandas as pd
# Cargar el resultado de la Tarea 5B
# Iterar sobre la lista de tuplas
# Crear el DataFrame
# Exportar el DataFrame a Excel

In [ ]:
import os

from google.colab import drive

# Acceso a través de Google Drive
drive.mount('/content/drive')

# Indicamos la ruta de la carpeta en Google Drive con nuestros textos del BOE y de EurLex
path = "/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/"

In [ ]:
# Creamos una lista para almacenar los datos recolectados
data = []

# Recorremos todos los archivos en el directorio
for archivo in os.listdir(path):
    if archivo.endswith(".txt"):
        archivo_path = os.path.join(path, archivo)
        with open(archivo_path, "r") as file:
            texto = file.readlines()
            for linea in texto:
                linea = linea.strip()
                if linea:  # Ignoramos líneas vacías
                    oracion, dict_entidades = eval(linea)
                    lista_entidades = dict_entidades.get('entities', [])
                    for tupla_entidades in lista_entidades:
                        texto_entidad, inicio, fin, tipo_entidad = tupla_entidades
                        data.append((oracion, texto_entidad, tipo_entidad, inicio, fin))

# Creamos el DataFrame de pandas
df = pd.DataFrame(data, columns=["Texto de la oración", "Texto de la entidad detectada", "Tipo de la entidad", "Posición_inicio", "Posición_fin"])

# Mostrar el DataFrame
df

,Texto de la oración,Texto de la entidad detectada,Tipo de la entidad,Posición_inicio,Posición_fin
0,Avis juridique important Acuerdos multilateral...,GATT,ORG,108,112
1,Avis juridique important Acuerdos multilateral...,Comité de Negociaciones Comerciales,ORG,998,1033
2,"En tales casos , un país en desarrollo Parte e...",Parte,ORG,1860,1865
3,"En tales casos , un país en desarrollo Parte e...",Acuerdo,ORG,2036,2043
4,Si los países en desarrollo formulan esa reser...,Acuerdo,ORG,3090,3097
...,...,...,...,...,...
4114,REPÚBLICA DEL SENEGAL un Pueblo - una Meta - u...,OCEANOGRAFÍA,ORG,37126,37138
4115,Consignatario : ... Número de licencia : ... P...,REPÚBLICA,ORG,39728,39737
4116,Consignatario : ... Número de licencia : ... P...,OCEANOGRAFÍA,ORG,39794,39806
4117,El tribunal de arbitraje adoptará sus decision...,Acuerdo,ORG,41541,41548


In [ ]:
df_legal = df[df['Tipo de la entidad'] == 'LEGAL']

df_legal.to_excel("/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/tabla_legal.xlsx", index=False)

In [ ]:
df_legal[:10]

,Texto de la oración,Texto de la entidad detectada,Tipo de la entidad,Posición_inicio,Posición_fin
275,Si Albania solicita ayuda externa de la Comuni...,"Reglamento (CE) no 2666/2000 del Consejo, de 5...",LEGAL,64,617
280,Artículo 6 El Memorándum de Acuerdo estipulará...,"Reglamento financiero de la Comunidad, que el ...",LEGAL,70,275
287,"Artículo 9 El Acuerdo se aplicará, por una par...",Tratado constitutivo de la Comunidad Europea y...,LEGAL,97,194
300,"B r n Bruksel, n dat nj zet e dy n ntor t viti...","Decisión 2000/750/CE del Consejo, de 27 de nov...",LEGAL,1942,2111
304,[2] Véase la Decisión no 50/2002/CE del Parlam...,Decisión no 50/2002/CE del Parlamento Europeo ...,LEGAL,13,256
307,[3] Véase la Decisión no 20/2004/CE del Parlam...,Decisión no 20/2004/CE del Parlamento Europeo ...,LEGAL,13,256
311,[4] Véase la Decisión no 791/2004/CE del Parla...,Decisión no 791/2004/CE del Parlamento Europeo...,LEGAL,13,295
313,"[5] Véase la Decisión 2004/100/CE del Consejo,...","Decisión 2004/100/CE del Consejo, de 26 de ene...",LEGAL,13,205
317,[6] Véase la Decisión no 1786/2002/CE del Parl...,Decisión no 1786/2002/CE del Parlamento Europe...,LEGAL,13,210
324,[7] Véase la Decisión no 508/2000/CE del Parla...,Decisión no 508/2000/CE del Parlamento Europeo...,LEGAL,13,152


In [ ]:
df_org = df[df['Tipo de la entidad'] == 'ORG']

df_org.to_excel("/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/tabla_org.xlsx", index=False)

In [ ]:
df_org[:10]

,Texto de la oración,Texto de la entidad detectada,Tipo de la entidad,Posición_inicio,Posición_fin
0,Avis juridique important Acuerdos multilateral...,GATT,ORG,108,112
1,Avis juridique important Acuerdos multilateral...,Comité de Negociaciones Comerciales,ORG,998,1033
2,"En tales casos , un país en desarrollo Parte e...",Parte,ORG,1860,1865
3,"En tales casos , un país en desarrollo Parte e...",Acuerdo,ORG,2036,2043
4,Si los países en desarrollo formulan esa reser...,Acuerdo,ORG,3090,3097
5,Si los países en desarrollo formulan esa reser...,Acuerdo,ORG,3602,3609
6,Las Partes en el Acuerdo convienen en que si e...,Acuerdo,ORG,3963,3970
7,Las Partes en el Acuerdo convienen en que si e...,Acuerdo,ORG,4079,4086
8,"Reconocen que las Partes en el Acuerdo , con s...",Acuerdo,ORG,4838,4845
9,"Está abierto a la aceptación , mediante firma ...",Acuerdo,ORG,5796,5803


## Comentarios sobre las entidades 'LEGAL'

Comprobamos que hay 2 entidades que no se reconocen bien
1. Reglamentos: solo había establecido una expresión regular para que abarcase desde "Reglamento" hasta el punto siguiente; sin embargo, en muchas oraciones vemos que sigue la misma estructura que "Decisión" e incluye un "(DO L 308...)". Para solucionarlo, añadimos esta opción como posible parada.
2. Tratados: solamente hay un tipo de tratado y es el "Tratado constitutivo de la Comunidad Europea o Tratado constitutivo de la Comunidad Económica Europea. Limitamos la regex a estas dos opciones.

In [ ]:
import re

def extraer_nuevas_leyes(textos_tokenizados):
    # Patrones para reconocer leyes, decisiones, reglamentos, tratados y órdenes
    patron_ley = re.compile(r'\b(?:Ley|Real\sDecreto)\b[^,]*(?:,\s*de[^,]*)?(?:,\s*(?:(?!se\s(publican|publica|anuncian|anuncia|notifican|notifica|comunican|comunica|informan|informa)\b)[^,])*)?')
    patron_decision = re.compile(r"Decisión.*?(?=\(DO|$)")
    patron_reglamento = re.compile(r"Reglamento.*?(?=[\.\(]DO|\.$)")
    patron_tratado = re.compile(r"(Tratado constitutivo de la Comunidad Europea|Tratado constitutivo de la Comunidad Económica Europea)")
    patron_orden = re.compile(r"Orden\s(?:[A-Z\d\/]+(?:\/\d{4})?|(?:de\s)?\d{1,2}\sde\s[a-zA-Z]+\sde\s\d{4})")

    # Lista de tuplas para almacenar resultados
    lista_tuplas_leyes = []

    # Iteramos sobre textos tokenizados
    for doc in textos_tokenizados:
        for sent in doc.sents:
            oracion = sent.text
            leyes_encontradas = patron_ley.findall(oracion)
            decision_encontrada = patron_decision.findall(oracion)
            reglamento_encontrado = patron_reglamento.findall(oracion)
            tratado_encontrado = patron_tratado.findall(oracion)
            orden_encontrada = patron_orden.findall(oracion)

            if leyes_encontradas or decision_encontrada or reglamento_encontrado or tratado_encontrado or orden_encontrada:
                dict_entities = {"entities": [(ley, oracion.find(ley), oracion.find(ley) + len(ley), 'LEGAL') for ley in leyes_encontradas]}
                dict_entities["entities"].extend([(dec, oracion.find(dec), oracion.find(dec) + len(dec), 'LEGAL') for dec in decision_encontrada])
                dict_entities["entities"].extend([(reg, oracion.find(reg), oracion.find(reg) + len(reg), 'LEGAL') for reg in reglamento_encontrado])
                dict_entities["entities"].extend([(tra, oracion.find(tra), oracion.find(tra) + len(tra), 'LEGAL') for tra in tratado_encontrado])
                dict_entities["entities"].extend([(ord, oracion.find(ord), oracion.find(ord) + len(ord), 'LEGAL') for ord in orden_encontrada])

                tupla_resultado = (oracion, dict_entities)
                lista_tuplas_leyes.append(tupla_resultado)

    return lista_tuplas_leyes


In [ ]:
# Llamamos a la función
leyes_en_textos = extraer_nuevas_leyes(textos_tokenizados)

In [ ]:
# Vemos el número de entidades totales

print(f"Número Total de Entidades LEGAL: {len(leyes_en_textos)}")

Número Total de Entidades LEGAL: 281


## Comentarios sobre las entidades 'ORG'

In [ ]:
# Como hay demasiadas entidades incorrectas, la manera más rápida que he encontrado de solucionar este probemos es pasar una lista de entidades
# que no queremos que reconozca como ORG a la función que creamos en la actividad anterior.

In [ ]:
# Cojo el texto de la primera oración con entidades mal reconocidas para hacer la prueba

texto_ORG = """Avis juridique important Convenio relativo a la conservación de la vida silvestre y del medio natural de Europa Diario Oficial n L 038 de 10/02/1982 p. 0003 - 0032 Edición especial en finés : Capítulo 11 Tomo 10 p. 0089 Edición especial sueca: Capítulo 11 Tomo 10 p. 0089 Edición especial en español: Capítulo 15 Tomo 3 p. 0086 Edición especial en portugués: Capítulo 15 Tomo 3 p. 0086 CONVENIO relativo a la conservación de la vida silvestre y del medio natural de Europa PREÁMBULO LOS ESTADOS MIEMBROS DEL CONSEJO DE EUROPA Y LOS DEMÁS SIGNATARIOS DEL PRESENTE CONVENIO , Considerando que el fin del Consejo de Europa es la realización de una unión más íntima entre sus miembros ; Teniendo en cuenta la voluntad del Consejo de Europa de cooperar con otros Estados en el campo de la conservación de la naturaleza ; Reconociendo que la flora y la fauna silvestres constituyen un patrimonio natural de un valor intrínseco , económico , recreativo , cultural , científico y estético que importa preservar y transmitir a las generaciones futuras ; Reconociendo , asimismo , el papel esencial de la flora y de la fauna silvestres en el mantenimiento de los equilibrios biológicos ; Constatando la rarefacción de muchas especies de la flora y de la fauna silvestres y la amenaza de extinción que pesa sobre alguna de ellas ; Conscientes de que la conservación de los hábitat naturales es uno de los factores esenciales para la protección y la preservación de la flora y de la fauna silvestres ; Reconociendo que la conservación de la flora y de la fauna silvestres debería tomarse en consideración por los gobiernos , en sus objetivos y programas nacionales , y que debería establecerse una cooperación internacional con el fin de proteger concretamente las especies migratorias ; Conscientes de que existen peticiones de medidas comunes , procedentes de gobiernos o de instancias internacionales , concretamente las hechas por la Conferencia de las Naciones Unidas sobre el medio ambiente , de 1972 , y la Asamblea Consultiva del Consejo de Europa ; Queriendo particularmente seguir , en el campo de la conservación de la vida silvestre , las recomendaciones de la Resolución n 2 de la segunda Conferencia ministerial europea sobre el medio ambiente , CONVIENEN LO SIGUIENTE : CAPÍTULO I Disposiciones generales Artículo 1 1.
Artículo 8 Si se trata de la captura o muerte de las especies de fauna silvestre enumeradas en el Anexo III , y en los casos en que se hagan excepciones con arreglo al artículo 9 en lo que respecta a las especies enumeradas en el Anexo II , las Partes contratantes prohibirán la utilización de todos los medios no selectivos de captura y muerte y de los medios que puedan causar localmente la desaparición o turbar seriamente la tranquilidad de las poblaciones de una especie , en particular de los medios enumerados en el Anexo IV.
Además de las medidas indicadas en los artículos 4 , 6 , 7 y 8 , las Partes contratantes se obligan a coordinar sus esfuerzos para la conservación de las especies migratorias enumeradas en los Anexos I y II y cuya área de distribución se extienda por sus territorios.
En la aplicación de las disposiciones del presente Convenio , las Partes contratantes se obligan a : a ) Cooperar cada vez que resulte útil hacerlo , especialmente cuando dicha cooperación pudiera aumentar la eficacia de las medidas adoptadas con arreglo a otros artículos del presente Convenio ; b ) fomentar y coordinar los trabajos de investigación en relación con las finalidades del presente Convenio ; 2 .
"POLYGONACEAE : Rheum rhaponticum L. PRIMULACEAE : Primula apennina Widmer."""

In [ ]:
# Aquí cargo el archivo con la lista de entidades que he anotado para que no me devuelva

def cargar_palabras_desde_archivo(archivo):
    with open(archivo, "r", encoding="utf-8") as file:
        contenido = file.read()  # Leer todo el contenido del archivo
        palabras = contenido.splitlines()  # Dividir el contenido en líneas y almacenarlo en una lista
    return palabras

# Uso de la función
nombre_archivo = "org_corregidas.txt"  # Nombre del archivo que contiene las palabras
no_entidades_org = cargar_palabras_desde_archivo(nombre_archivo)  # Almacenar las palabras en una lista
print(no_entidades_org)

['"Real Decreto", "EUROPA", "Resolución", "las Partes", "las Partes", "Convenio , las Partes", "Comité de Ministros del", "Anexos", "Secretario General del Consejo de Europa", "Artículo 24 El Secretario General del Consejo de Europa", "Degen & Baldacci Symphytum", "CARYOPHYLLACEAE", "Tuberaric major", "Martinovsky & H. Scholz GROSSULARIACEAE", "& Auch", "PRIMULACEAE", "UMBELLIFERAE", "Talpidae Desmana pyrenaica", "Felidae Lynx", "Phocaenidae Phocaena", "Melanocorupha", "Melanocorypha", "Emberizidae", "cia", "Vipera ursinii Vipera latasti Vipera ammodytes Vipera xanthina", "UCRANIA", "CONSCIENTES", "HAN CONVENIDO", "Artículo 1 A fin de cumplir", "Artículo 10 El Ministerio de Asuntos Exteriores de Ucrania", "FE DE LO CUAL", "República de Albania", "ALBANIA", "Proceso", "HAN", "CONVENIDO", "República Yugoslava de Macedonia", "Fait Bruxelles", "jum ta\' Novembru", "dy n ntor t vitit", "Comunidad Europea Za Evropské", "For Det Europ iske F", "Republik s s Shqip ris", "COMUNITARIOS", "Juvent

In [ ]:
# Cargamos el modelo de spaCy

def extraer_nuevas_ORG(textos_tokenizados):
    lista_tuplas_entidades = []

    for doc in textos_tokenizados:
        for sent in doc.sents:
            texto_sent = sent.text
            dict_entities = {}

            if sent.ents:
                lista_tupla_ne_spacy = []

                for ent in sent.ents:
                    if ent.label_ == "ORG" and not any(ent.text in s for s in no_entidades_org):
                        tupla_ne_spacy = (ent.text, ent.start_char, ent.end_char, ent.label_)
                        lista_tupla_ne_spacy.append(tupla_ne_spacy)

                dict_entities["entities"] = lista_tupla_ne_spacy
                tupla_oración_entidades = (sent.text, dict_entities)
                lista_tuplas_entidades.append(tupla_oración_entidades)

    return lista_tuplas_entidades

In [ ]:
# Tokenizamos el texto ptra vez para poder pasarle la función y que nos lo devuelva con el formato que queremos
texto_tokenizado = nlp(texto_ORG)

# Llamamos a la función con el documento de prueba tokenizado
resultado = extraer_nuevas_ORG([texto_tokenizado])

print(resultado)

[('Avis juridique important Convenio relativo a la conservación de la vida silvestre y del medio natural de Europa Diario Oficial n L 038 de 10/02/1982 p. 0003 - 0032 Edición especial en finés : Capítulo 11 Tomo 10 p. 0089 Edición especial sueca: Capítulo 11 Tomo 10 p. 0089 Edición especial en español: Capítulo 15 Tomo 3 p. 0086 Edición especial en portugués: Capítulo 15 Tomo 3 p. 0086 CONVENIO relativo a la conservación de la vida silvestre y del medio natural de Europa PREÁMBULO LOS ESTADOS MIEMBROS DEL CONSEJO DE EUROPA Y LOS DEMÁS SIGNATARIOS DEL PRESENTE CONVENIO , Considerando que el fin del Consejo de Europa es la realización de una unión más íntima entre sus miembros ; Teniendo en cuenta la voluntad del Consejo de Europa de cooperar con otros Estados en el campo de la conservación de la naturaleza ; Reconociendo que la flora y la fauna silvestres constituyen un patrimonio natural de un valor intrínseco , económico , recreativo , cultural , científico y estético que importa pres

In [ ]:
# Vamos a sacar las entidades de todos los textos

resultados_org = extraer_nuevas_ORG(textos_tokenizados)

In [ ]:
# Vamos a visualizar los datos de una manera más estética usando pandas

import pandas as pd

columnas = ["Oración", "Entidad", "Comienzo car.", "Fin car.", "Tipo"]
lista_resultados_procesados = []

for idx, (oracion, entidades) in enumerate(resultados_org):
    for entidad in entidades["entities"]:
        lista_resultados_procesados.append((oracion, entidad[0], entidad[1], entidad[2], entidad[3]))

# Creamos DataFrame en pandas
df_resultados_org = pd.DataFrame(lista_resultados_procesados, columns=columnas)

# Imprimimos DataFrame
df_resultados_org

,Oración,Entidad,Comienzo car.,Fin car.,Tipo
0,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Acuerdo Europeo,57,72,ORG
1,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Acuerdo Europeo,641,656,ORG
2,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Acuerdo Europeo,1225,1240,ORG
3,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,Unión Europea de la República Checa,1421,1456,ORG
4,JRC-ACQUIS 22005A0617(01) Spanish Protocolo ad...,ALEMANIA,1769,1777,ORG
...,...,...,...,...,...
1577,a ) El presente Acuerdo estará abierto a la ac...,Comunidad Económica Europea,87277,87304,ORG
1578,Los servicios de Secretaría del presente Acuer...,Secretaría del GATT,91199,91218,ORG
1579,El presente Acuerdo será depositado en poder d...,Partes Contratantes del Acuerdo General,91411,91450,ORG
1580,El presente Acuerdo será registrado de conform...,Carta de las Naciones Unidas,91853,91881,ORG


In [ ]:
# Vemos el número de entidades totales

print(f"Número Total de Entidades ORG: {len(df_resultados_org)}")

Número Total de Entidades ORG: 1582


In [ ]:
def combinar_entidades(org_entidades, ley_entidades):
    # Creamos un diccionario para organizar las entidades por oración
    oraciones_entidades = {}

    # Agregamos entidades ORG al diccionario
    for oracion, entidades in org_entidades:
        oraciones_entidades.setdefault(oracion, {"entities": []})["entities"].extend(
            [(ent[1], ent[2], ent[3]) for ent in entidades.get("entities", [])]
        )

    # Agregamos entidades de ley al diccionario
    for oracion, entidades in ley_entidades:
        oraciones_entidades.setdefault(oracion, {"entities": []})["entities"].extend(
            [(ent[1], ent[2], ent[3]) for ent in entidades["entities"]]
        )

    # Eliminamos las oraciones que no tienen entidades
    oraciones_entidades = {oracion: entidades for oracion, entidades in oraciones_entidades.items() if entidades["entities"]}

    # Convertimos el diccionario a una lista de tuplas
    lista_tuplas_resultado = [(oracion, {"entities": entidades["entities"]}) for oracion, entidades in oraciones_entidades.items()]

    return lista_tuplas_resultado  # No hay necesidad de envolverlo en una lista adicional

In [ ]:
# Llamamos a la función para combinar entidades ORG y LEGAL
entidades_combinadas = combinar_entidades(resultados_org, leyes_en_textos)

# Imprimimos resultados
print("Lista de entidades combinadas:")
print(entidades_combinadas)  # No es necesario acceder al índice [0]

Lista de entidades combinadas:
[('JRC-ACQUIS 22005A0617(01) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y la Rep%uacute%blica de Bulgaria, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Ch

In [ ]:
# Ruta donde se guardarán los archivos anotados
path = "/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/"

# Compilamos los patrones regex
patron_ley = re.compile(r'\b(?:Ley|Real\sDecreto)\b[^,]*(?:,\s*de[^,]*)?(?:,\s*(?:(?!se\s(publican|publica|anuncian|anuncia|notifican|notifica|comunican|comunica|informan|informa)\b)[^,])*)?')
patron_decision = re.compile(r"Decisión.*?(?=\(DO|$)")
patron_reglamento = re.compile(r"Reglamento.*?(?=\.|$)")
patron_tratado = re.compile(r"Tratado constitutivo.*?(?=,|$)")
patron_orden = re.compile(r"Orden\s(?:[A-Z\d\/]+(?:\/\d{4})?|(?:de\s)?\d{1,2}\sde\s[a-zA-Z]+\sde\s\d{4})")

# Iteramos sobre cada documento tokenizado de la lista de textos tokenizados
for i, doc in enumerate(textos_tokenizados, start=1):
    # Creamos una lista para almacenar las tuplas de cada línea
    lineas_anotadas = []

    # Iteramos sobre cada oración del documento
    for sent in doc.sents:
        # Texto de la oración
        texto_oracion = sent.text

        # Buscamos todas las coincidencias de cada patrón en la oración actual
        matches_legales = re.finditer(patron_ley, texto_oracion)
        matches_decision = re.finditer(patron_decision, texto_oracion)
        matches_reglamento = re.finditer(patron_reglamento, texto_oracion)
        matches_tratado = re.finditer(patron_tratado, texto_oracion)
        matches_orden = re.finditer(patron_orden, texto_oracion)

        # Lista para almacenar todas las entidades encontradas en la oración actual
        entidades_oracion = []

        # Iteramos sobre las coincidencias encontradas para cada patrón
        for matches, etiqueta in [(matches_legales, "LEGAL"), (matches_decision, "LEGAL"), (matches_reglamento, "LEGAL"), (matches_tratado, "LEGAL"), (matches_orden, "LEGAL")]:
            for match in matches:
                entidad_texto = match.group()
                inicio, fin = match.span()
                entidades_oracion.append((entidad_texto, inicio, fin, etiqueta))

        # Obtenemos las entidades ORG de la oración actual usando la función proporcionada
        entidades_org = anotar_spaCy_ORG([sent])

        # Verificamos si se encontraron entidades ORG en la oración
        if entidades_org:
            # Agregamos las entidades ORG de la oración actual a la lista de datos de entrenamiento
            lineas_anotadas.append(entidades_org[0])

        # Verificamos si se encontraron entidades en la oración
        if entidades_oracion:
            # Ordenamos las entidades por su posición de inicio
            entidades_oracion.sort(key=lambda x: x[1])
            # Agregamos las entidades encontradas de la oración actual a la lista de datos de entrenamiento
            lineas_anotadas.append((texto_oracion, {"entities": [(entidad_texto, inicio, fin, etiqueta) for entidad_texto, inicio, fin, etiqueta in entidades_oracion]}))

    # Guardamos las líneas anotadas en un nuevo archivo .txt
    nombre_archivo = os.path.join(path, f"doc_{i}_anotado.txt")
    with open(nombre_archivo, 'w', encoding="utf-8") as archivo:
        # Escribimos las tuplas en el archivo
        for linea_anotada in lineas_anotadas:
            archivo.write(f"{linea_anotada}\n")

    print(f"Se ha creado el archivo '{nombre_archivo}'.")

Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_1_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_2_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_3_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_4_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_5_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_6_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_7_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_8_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicaciones/Doaa/Textos_Anotaciones/doc_9_anotado.txt'.
Se ha creado el archivo '/content/drive/MyDrive/Aplicac

# Paso 4: Entrenamiento

In [ ]:
# Importar los módulos necesarios
import spacy
import random
from spacy.training.example import Example
import pandas as pd

In [ ]:
def filter_overlapping_spans(entities):
    filtered_entities = []
    entities = sorted(entities, key=lambda x: x[0])
    i = 0
    while i < len(entities):
        start, end, label = entities[i]
        j = i + 1
        while j < len(entities) and entities[j][0] < end:
            if entities[j][1] > end:
                end = entities[j][1]
            j += 1
        filtered_entities.append((start, end, label))
        i = j
    return filtered_entities

def entrenar(datos, iteraciones):
    datos_gold = datos
    nlp = spacy.blank('es')
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)

    for _, anns in datos_gold:
        for ent in anns.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.select_pipes(disable=other_pipes):
        optimizer = nlp.begin_training()
        for n in range(iteraciones):
            print("Iteración número " + str(n))
            random.shuffle(datos_gold)
            losses = {}
            for texto, anns in datos_gold:
                doc = nlp.make_doc(texto)
                golds = []
                for ent in anns.get('entities'):
                    start, end, label = ent[0:3]
                    golds.append((start, end, label))
                golds = filter_overlapping_spans(golds)
                ejemplo = Example.from_dict(doc, {"entities": golds})
                nlp.update(
                    [ejemplo],
                    drop=0.2,
                    sgd=optimizer,
                    losses=losses
                )
            print(losses)
    return nlp

In [ ]:
# Anotar los ejemplos a validar
ner_legal = entrenar(entidades_combinadas, 30)
ner_legal.to_disk("modelo_ner_legal_prueba")

Iteración número 0
{'ner': 1043.526180453013}
Iteración número 1
{'ner': 675.6956603064938}
Iteración número 2
{'ner': 487.3365187132495}
Iteración número 3
{'ner': 415.05730802879765}
Iteración número 4
{'ner': 476.2376782204264}
Iteración número 5
{'ner': 667.3538369841243}
Iteración número 6
{'ner': 404.55360760574695}
Iteración número 7
{'ner': 469.1133253049289}
Iteración número 8
{'ner': 465.1853340792771}
Iteración número 9
{'ner': 445.0027722074467}
Iteración número 10
{'ner': 641.8358616102194}
Iteración número 11
{'ner': 544.8269361921176}
Iteración número 12
{'ner': 859.9855671489794}
Iteración número 13
{'ner': 397.6306446920136}
Iteración número 14
{'ner': 591.0030445271309}
Iteración número 15
{'ner': 797.4474436720321}
Iteración número 16
{'ner': 422.9851763467125}
Iteración número 17
{'ner': 529.6629167327551}
Iteración número 18
{'ner': 599.5890414683499}
Iteración número 19
{'ner': 466.17258122368827}
Iteración número 20
{'ner': 419.5312368680131}
Iteración número 21


In [ ]:
# Guardamos el modelo creado:
ruta = '/content/drive/MyDrive/Aplicaciones/Doaa/modelo_trained/'

In [ ]:
ner_legal.to_disk(ruta)

# Paso 5: Evaluación

In [ ]:
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [ ]:
carpeta_test = "/content/drive/MyDrive/Aplicaciones/Doaa/Test/"
textos_test = reescribir_textos_normalizados(carpeta_test)

In [ ]:
# Imprimimos el primer texto normalizado
if textos_test:
    texto_test_1 = textos_test[0]
    print("Primer texto normalizado:")
    print(texto_test_1)
else:
    print("No se ha extraído ningún texto normalizado.")

Primer texto normalizado:
JRC-ACQUIS 22005A0617(02) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y Ruman%iacute%a, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y Ruman%iacute%a, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%

In [ ]:
textos_token_test = []

# Procesamos cada texto normalizado pasándole el modelo de spaCy y lo añadimos a la lista de textos tokenizados
for texto_test in textos_test:
    # Le pasamos spaCy
    doc = nlp(texto_test)
    # Añadimos los documentos que hemos procesado en doc a la lista de textos tokenizados
    textos_token_test.append(doc)

In [ ]:
for i, doc in enumerate(textos_token_test, start=1):
    # Verificar si el texto ha superado las 100 palabras
    palabras_contadas = 0
    # Inicializamos una lista para almacenar las palabras
    palabras = []
    # Iteramos sobre los tokens
    for token in doc:
        # Añadir la palabra actual a la lista de palabras
        palabras.append(token.text)
        palabras_contadas += 1
        # Si hemos alcanzado las 100 palabras, salimos del bucle
        if palabras_contadas >= 100:
            break
    # Unir las primeras 100 palabras en un solo string
    primeras_100_palabras = " ".join(palabras)
    print("Texto", i, primeras_100_palabras)

Texto 1 JRC-ACQUIS 22005A0617(02 ) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros , por una parte , y Ruman%iacute%a , por otra , para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa , la Rep%uacute%blica de Estonia , la Rep%uacute%blica de Chipre , la Rep%uacute%blica de Hungr%iacute%a , la Rep%uacute%blica de Letonia , la Rep%uacute%blica de Lituania , la Rep%uacute%blica de Malta , la Rep%uacute%blica de Polonia , la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el
Texto 2 LISTA DE SUSTANCIAS Y MÉTODOS PROHIBIDOS DE 2018 CÓDIGO MUNDIAL ANTIDOPAJE Fecha de entrada en vigor : 1 de enero de 2018 Sustancias y métodos prohibidos en todo momento ( en y fuera de competición ) En virtud de lo dispuesto en el artículo 4.2.2 del Código Mundial Antidopaje , todas las sustancias prohibidas deberán consi

In [ ]:
org_test = anotar_spaCy_ORG(textos_token_test)
leyes_test = extraer_leyes(textos_token_test)

In [ ]:
# Vamos a visualizar los datos de una manera más estética usando pandas

import pandas as pd

columnas = ["Oración", "Entidad", "Comienzo car.", "Fin car.", "Tipo"]
lista_resultados_procesados = []

for idx, (oracion, entidades) in enumerate(org_test):
    for entidad in entidades["entities"]:
        lista_resultados_procesados.append((oracion, entidad[0], entidad[1], entidad[2], entidad[3]))

# Creamos DataFrame en pandas
df_org_test = pd.DataFrame(lista_resultados_procesados, columns=columnas)

# Imprimimos DataFrame
df_org_test

,Oración,Entidad,Comienzo car.,Fin car.,Tipo
0,JRC-ACQUIS 22005A0617(02) Spanish Protocolo ad...,Acuerdo Europeo,57,72,ORG
1,JRC-ACQUIS 22005A0617(02) Spanish Protocolo ad...,Acuerdo Europeo,624,639,ORG
2,JRC-ACQUIS 22005A0617(02) Spanish Protocolo ad...,Acuerdo Europeo,1191,1206,ORG
3,JRC-ACQUIS 22005A0617(02) Spanish Protocolo ad...,Unión Europea de la República Checa,1370,1405,ORG
4,JRC-ACQUIS 22005A0617(02) Spanish Protocolo ad...,ALEMANIA,1718,1726,ORG
...,...,...,...,...,...
204,"Unio ravoisieri, (antes U. elongatulus).Náyade...",Unio,12215,12219,ORG
205,Boraginaceae.,Boraginaceae,12440,12452,ORG
206,"Pedicularis comosa, subsp.",Pedicularis,12740,12751,ORG
207,Ranunculaceae.,Ranunculaceae,12960,12973,ORG


In [ ]:
print(f"Número Total de Entidades ORG: {len(df_org_test)}")
print(f"Número Total de Entidades LEGAL: {len(leyes_test)}")

Número Total de Entidades ORG: 209
Número Total de Entidades LEGAL: 42


In [ ]:
def combinar_entidades(org_entidades, ley_entidades):
    # Creamos un diccionario para organizar las entidades por oración
    oraciones_entidades = {}

    # Agregamos entidades ORG al diccionario
    for oracion, entidades in org_entidades:
        oraciones_entidades.setdefault(oracion, {"entities": []})["entities"].extend(
            [(ent[1], ent[2], ent[3]) for ent in entidades.get("entities", [])]
        )

    # Agregamos entidades de ley al diccionario
    for oracion, entidades in ley_entidades:
        oraciones_entidades.setdefault(oracion, {"entities": []})["entities"].extend(
            [(ent[1], ent[2], ent[3]) for ent in entidades["entities"]]
        )

    # Eliminamos las oraciones que no tienen entidades
    oraciones_entidades = {oracion: entidades for oracion, entidades in oraciones_entidades.items() if entidades["entities"]}

    # Convertimos el diccionario a una lista de tuplas
    lista_tuplas_resultado = [(oracion, {"entities": entidades["entities"]}) for oracion, entidades in oraciones_entidades.items()]

    return lista_tuplas_resultado  # No hay necesidad de envolverlo en una lista adicional

# Llamamos a la función para combinar entidades ORG y LEGAL
combinadas_test = combinar_entidades(org_test, leyes_test)

# Imprimimos resultados
print("Lista de entidades combinadas:")
print(combinadas_test)

Lista de entidades combinadas:
[('JRC-ACQUIS 22005A0617(02) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y Ruman%iacute%a, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y Ruman%iacute%a, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr

In [ ]:
def my_evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annotations in examples:
        pred = ner_model(input_)
        print(pred,annotations)
        temp = Example.from_dict(pred, dict.fromkeys(annotations))
        example.append(temp)
    scores = scorer.score(example)
    return scores

In [ ]:
import spacy

ner_model = spacy.load('/content/drive/MyDrive/Aplicaciones/Doaa/modelo_trained/') # indicar la ruta del modelo entrenado
results = my_evaluate(ner_model, combinadas_test)

print(results)

JRC-ACQUIS 22005A0617(02) Spanish Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y Ruman%iacute%a, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de Letonia, la Rep%uacute%blica de Lituania, la Rep%uacute%blica de Malta, la Rep%uacute%blica de Polonia, la Rep%uacute%blica de Eslovenia y la Rep%uacute%blica Eslovaca Protocolo adicional al Acuerdo Europeo por el que se crea una asociaci%oacute%n entre las Comunidades Europeas y sus Estados miembros, por una parte, y Ruman%iacute%a, por otra, para tener en cuenta la adhesi%oacute%n a la Uni%oacute%n Europea de la Rep%uacute%blica Checa, la Rep%uacute%blica de Estonia, la Rep%uacute%blica de Chipre, la Rep%uacute%blica de Hungr%iacute%a, la Rep%uacute%blica de 